In [48]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio

#### Testing File Paths

In [49]:
CAPUCHIN_FILE = os.path.join('data', 'Parsed_Capuchinbird_Clips', 'XC3776-0.wav')
NOT_CAPUCHIN_FILE = os.path.join('data', 'Parsed_Not_Capuchinbird_Clips', 'afternoon-birds-song-in-forest-0.wav')


In [51]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

wave = load_wav_16k_mono(CAPUCHIN_FILE)
plt.plot(wave)
plt.show()

NotImplementedError: in user code:

    File "/Users/kyle-anthonyhay/Documents/Code/Python/Jupyter_Notebooks/Bird_Sound_Recognizer/Bird-Call-Identifier/venv/lib/python3.10/site-packages/tensorflow_io/python/ops/audio_ops.py", line 465, in f
        return core_ops.io_audio_resample(
    File "/Users/kyle-anthonyhay/Documents/Code/Python/Jupyter_Notebooks/Bird_Sound_Recognizer/Bird-Call-Identifier/venv/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py", line 88, in __getattr__
        return getattr(self._load(), attrb)
    File "/Users/kyle-anthonyhay/Documents/Code/Python/Jupyter_Notebooks/Bird_Sound_Recognizer/Bird-Call-Identifier/venv/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py", line 84, in _load
        self._mod = _load_library(self._library)
    File "/Users/kyle-anthonyhay/Documents/Code/Python/Jupyter_Notebooks/Bird_Sound_Recognizer/Bird-Call-Identifier/venv/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py", line 69, in _l